## RE Demo 
## Preprocessing Data Input Tables Regulatory Overlay Areas RShiny Demo App
Notes:
- asdf

In [1]:
# working with data
import os
import numpy as np
import pandas as pd
import geopandas as gpd # the library that lets us read in shapefiles
import geoplot as gplt # for plotting maps

# visulizaiton
import seaborn as sns
import matplotlib.pyplot as plt

# Cleanup
from datetime import datetime
pd.set_option('display.max_columns', 999)  # How to display all columns of a Pandas DataFrame in Jupyter Notebook

In [2]:
# Working Directory and Input File
workingDir = "C:/Users/rjame/Documents/WSWC Documents/WaDE Side Projects Local/Rshiny RE Demo/Rshiny_RO_Map/NotEssentialData"
os.chdir(workingDir)

## Inputs

In [3]:
#reportingunits_regulatoryoverlays Input
fileInput = "reportingunits_regulatoryoverlays.csv"
dfruro = pd.read_csv(fileInput)
dfruro = dfruro.replace(np.nan, "").drop_duplicates().reset_index(drop=True)
print(len(dfruro))
dfruro.head()

2364


,StateCV,ReportingUnitUUID,ReportingUnitName,ReportingUnitNativeID,ReportingUnitTypeCV,RegulatoryName,RegulatoryDescription,RegulatoryStatusCV,OversightAgency,RegulatoryStatuteLink,StatutoryEffectiveDate,RegulatoryOverlayTypeCV,WaDENameRU,WaDENameRO,WaDENameWS
0,AZ,AZre_RUaz1,SANTA CRUZ AMA,az1,Active Management Areas,SANTA CRUZ AMA,"To safeguard the health, safety and economic w...",Active,Arizona Department of Water Resources,,1994-07-01,Active Management Areas,Custom - Medium,Regulatory,Groundwater
1,AZ,AZre_RUaz2,PRESCOTT AMA,az2,Active Management Areas,PRESCOTT AMA,"To safeguard the health, safety and economic w...",Active,Arizona Department of Water Resources,,1980-06-12,Active Management Areas,Custom - Medium,Regulatory,Groundwater
2,AZ,AZre_RUaz3,HARQUAHALA INA,az3,Irrigation Non-Expansion Areas,HARQUAHALA INA,"To safeguard the health, safety and economic w...",Active,Arizona Department of Water Resources,,1981-01-01,Irrigation Non-Expansion Areas,Irrigation Non-Expansion Areas,Regulatory,Groundwater
3,AZ,AZre_RUaz4,JOSEPH CITY INA,az4,Irrigation Non-Expansion Areas,JOSEPH CITY INA,"To safeguard the health, safety and economic w...",Active,Arizona Department of Water Resources,,1974-12-16,Irrigation Non-Expansion Areas,Irrigation Non-Expansion Areas,Regulatory,Groundwater
4,AZ,AZre_RUaz5,PHOENIX AMA,az5,Active Management Areas,PHOENIX AMA,"To safeguard the health, safety and economic w...",Active,Arizona Department of Water Resources,,1980-06-12,Active Management Areas,Custom - Medium,Regulatory,Groundwater


In [4]:
#reportingunit geometry
fileInput = "ru_geometry.csv"
dfrug = pd.read_csv(fileInput)
#dfrug['geometry'] = gpd.GeoSeries.from_wkt(dfrug['geometry'], crs="EPSG:4326")
print(len(dfrug))
dfrug.head()

2364


,ReportingUnitUUID,geometry
0,AZre_RUaz1,POLYGON ((-111.19920647185997 31.8582010611542...
1,AZre_RUaz2,POLYGON ((-112.56132756724693 34.7224554546705...
2,AZre_RUaz3,POLYGON ((-113.30171877245296 33.8349369910477...
3,AZre_RUaz4,POLYGON ((-110.27110877458067 34.9960090546926...
4,AZre_RUaz5,POLYGON ((-112.06643638934614 34.0000552467006...


## Clean Reporting Units & Regulatory Overlays Data

In [5]:
for x in dfruro['StateCV'].sort_values().unique():
    print(f'"' + x + '",')

"AZ",
"CA",
"CO",
"ID",
"KS",
"MT",
"ND",
"NE",
"NM",
"NV",
"OK",
"OR",
"SD",
"TX",
"WA",
"WY",


In [6]:
# Groupby ReportingUnitUUID
# create new dataframe: dfruro2
dfruro2 = dfruro.groupby(['ReportingUnitUUID']).agg(lambda x: ", ".join([str(elem) for elem in (list(set(x))) if elem!=""])).replace(np.nan, "").reset_index()
print(len(dfruro2))
dfruro2.head(1)

2364


,ReportingUnitUUID,StateCV,ReportingUnitName,ReportingUnitNativeID,ReportingUnitTypeCV,RegulatoryName,RegulatoryDescription,RegulatoryStatusCV,OversightAgency,RegulatoryStatuteLink,StatutoryEffectiveDate,RegulatoryOverlayTypeCV,WaDENameRU,WaDENameRO,WaDENameWS
0,AZre_RUaz1,AZ,SANTA CRUZ AMA,az1,Active Management Areas,SANTA CRUZ AMA,"To safeguard the health, safety and economic w...",Active,Arizona Department of Water Resources,,1994-07-01,Active Management Areas,Custom - Medium,Regulatory,Groundwater


In [7]:
# # polygon Color

# colorDictionary = {
#     "County" : "#648FFF", 
#     "HUC8" : "#785EF0",
#     "Custom - Small" : "#DC267F",
#     "Custom - Medium" : "#FE6100",
#     "Custom - Large" : "#FFB000"}

# def polygonColorFunc(Val):
#     Val = Val.strip()
#     try:
#         outString = colorDictionary[Val]
#     except:
#         outString = "???"
#     return outString

# dfru['polyColor'] = dfru.apply(lambda row: polygonColorFunc(row['WaDENameRU']), axis=1)
# for x in dfru['polyColor'].sort_values().unique():
#     print(f'"' + x + '",')

In [8]:
# Fixing blank WaDENameWS
def fixBlankValuesFunc(Val):
    Val = str(Val).strip()
    if Val == "" or pd.isnull(Val):
        outString = "WaDE Blank"
    else:
        outString = Val
    return outString

In [9]:
dfruro2['WaDENameWS'] = dfruro2.apply(lambda row: fixBlankValuesFunc(row['WaDENameWS']), axis=1)
for x in dfruro2['WaDENameWS'].sort_values().unique():
    print(f'"' + x + '",')

"Groundwater",
"Surface Water",
"Surface Water and or Groundwater",
"WaDE Blank",


In [10]:
dfruro2['WaDENameRO'] = dfruro2.apply(lambda row: fixBlankValuesFunc(row['WaDENameRO']), axis=1)
for x in dfruro2['WaDENameRO'].sort_values().unique():
    print(f'"' + x + '",')

"Administrative",
"Regulatory",
"WaDE Blank",


In [11]:
dfruro2['WaDENameRU'] = dfruro2.apply(lambda row: fixBlankValuesFunc(row['WaDENameRU']), axis=1)
for x in dfruro2['WaDENameRU'].sort_values().unique():
    print(f'"' + x + '",')

"Custom - Medium",
"Irrigation Non-Expansion Areas",
"WaDE Blank",


In [12]:
# attach geometry based on ReportingUnitUUID
dfruro2_rug = pd.merge(dfruro2, dfrug, on='ReportingUnitUUID', how='left')
dfruro2_rug = dfruro2_rug[dfruro2_rug['geometry'] != ""].reset_index(drop=True)
#dfruro2_rug = dfruro2_rug.dropna()
dfruro2_rug['geometry'] = gpd.GeoSeries.from_wkt(dfruro2_rug['geometry'], crs="EPSG:4326") # convert from string -to- geometry
print(len(dfruro2_rug))
dfruro2_rug.head(1)

2364


,ReportingUnitUUID,StateCV,ReportingUnitName,ReportingUnitNativeID,ReportingUnitTypeCV,RegulatoryName,RegulatoryDescription,RegulatoryStatusCV,OversightAgency,RegulatoryStatuteLink,StatutoryEffectiveDate,RegulatoryOverlayTypeCV,WaDENameRU,WaDENameRO,WaDENameWS,geometry
0,AZre_RUaz1,AZ,SANTA CRUZ AMA,az1,Active Management Areas,SANTA CRUZ AMA,"To safeguard the health, safety and economic w...",Active,Arizona Department of Water Resources,,1994-07-01,Active Management Areas,Custom - Medium,Regulatory,Groundwater,"POLYGON ((-111.19921 31.85820, -111.19889 31.8..."


In [13]:
# convert dataframe to geodatframe
gdfruro2 = gpd.GeoDataFrame(dfruro2_rug, geometry=dfruro2_rug['geometry'], crs="EPSG:4326") # covert to geodataframe
gdfruro2['geometry'] = gdfruro2.simplify(0.001) # simplify the geometry. Lower the number the larger the exported file.

# Review and Export out files.

In [14]:
gdfruro2.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2364 entries, 0 to 2363
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   ReportingUnitUUID        2364 non-null   object  
 1   StateCV                  2364 non-null   object  
 2   ReportingUnitName        2364 non-null   object  
 3   ReportingUnitNativeID    2364 non-null   object  
 4   ReportingUnitTypeCV      2364 non-null   object  
 5   RegulatoryName           2364 non-null   object  
 6   RegulatoryDescription    2364 non-null   object  
 7   RegulatoryStatusCV       2364 non-null   object  
 8   OversightAgency          2364 non-null   object  
 9   RegulatoryStatuteLink    2364 non-null   object  
 10  StatutoryEffectiveDate   2364 non-null   object  
 11  RegulatoryOverlayTypeCV  2364 non-null   object  
 12  WaDENameRU               2364 non-null   object  
 13  WaDENameRO               2364 non-null   object  
 14  

In [15]:
gdfruro2

,ReportingUnitUUID,StateCV,ReportingUnitName,ReportingUnitNativeID,ReportingUnitTypeCV,RegulatoryName,RegulatoryDescription,RegulatoryStatusCV,OversightAgency,RegulatoryStatuteLink,StatutoryEffectiveDate,RegulatoryOverlayTypeCV,WaDENameRU,WaDENameRO,WaDENameWS,geometry
0,AZre_RUaz1,AZ,SANTA CRUZ AMA,az1,Active Management Areas,SANTA CRUZ AMA,"To safeguard the health, safety and economic w...",Active,Arizona Department of Water Resources,,1994-07-01,Active Management Areas,Custom - Medium,Regulatory,Groundwater,"POLYGON ((-111.19921 31.85820, -111.19692 31.8..."
1,AZre_RUaz2,AZ,PRESCOTT AMA,az2,Active Management Areas,PRESCOTT AMA,"To safeguard the health, safety and economic w...",Active,Arizona Department of Water Resources,,1980-06-12,Active Management Areas,Custom - Medium,Regulatory,Groundwater,"POLYGON ((-112.56133 34.72246, -112.56106 34.7..."
2,AZre_RUaz3,AZ,HARQUAHALA INA,az3,Irrigation Non-Expansion Areas,HARQUAHALA INA,"To safeguard the health, safety and economic w...",Active,Arizona Department of Water Resources,,1981-01-01,Irrigation Non-Expansion Areas,Irrigation Non-Expansion Areas,Regulatory,Groundwater,"POLYGON ((-113.30172 33.83494, -113.29613 33.8..."
3,AZre_RUaz4,AZ,JOSEPH CITY INA,az4,Irrigation Non-Expansion Areas,JOSEPH CITY INA,"To safeguard the health, safety and economic w...",Active,Arizona Department of Water Resources,,1974-12-16,Irrigation Non-Expansion Areas,Irrigation Non-Expansion Areas,Regulatory,Groundwater,"POLYGON ((-110.27111 34.99601, -110.27062 34.9..."
4,AZre_RUaz5,AZ,PHOENIX AMA,az5,Active Management Areas,PHOENIX AMA,"To safeguard the health, safety and economic w...",Active,Arizona Department of Water Resources,,1980-06-12,Active Management Areas,Custom - Medium,Regulatory,Groundwater,"POLYGON ((-112.06644 34.00006, -112.06571 34.0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2359,WYre_RUwy5,WY,Crook County Natural Resource,wy5,Conservation Districts,Crook County Natural Resource,Selects conservation policy priorities which a...,Active,Wyoming Association of Conservation Districts,,1941-03-01,Conservation District,WaDE Blank,Administrative,Surface Water and or Groundwater,"POLYGON ((-105.08952 44.90938, -105.08939 45.0..."
2360,WYre_RUwy6,WY,Dubois-Crowheart Conservation,wy6,Conservation Districts,Dubois-Crowheart Conservation,Selects conservation policy priorities which a...,Active,Wyoming Association of Conservation Districts,,1941-03-01,Conservation District,WaDE Blank,Administrative,Surface Water and or Groundwater,"POLYGON ((-110.05383 43.90196, -110.05305 44.0..."
2361,WYre_RUwy7,WY,Hot Springs Conservation,wy7,Conservation Districts,Hot Springs Conservation,Selects conservation policy priorities which a...,Active,Wyoming Association of Conservation Districts,,1941-03-01,Conservation District,WaDE Blank,Administrative,Surface Water and or Groundwater,"POLYGON ((-109.31323 43.81215, -109.31308 43.8..."
2362,WYre_RUwy8,WY,Laramie County Conservation,wy8,Conservation Districts,Laramie County Conservation,Selects conservation policy priorities which a...,Active,Wyoming Association of Conservation Districts,,1941-03-01,Conservation District,WaDE Blank,Administrative,Surface Water and or Groundwater,"POLYGON ((-105.28070 41.39498, -105.27789 41.3..."


In [16]:
gdfruro2.to_file("reportingunits_regulatoryoverlays_v2.shp") # export shape file

AttributeError: module 'pandas' has no attribute 'Int64Index'

## Mapping to Validate Sites
- having a little issue getting geoplot to work with geopandas on geometry data

In [ ]:
# map the polygons
gplt.polyplot(gdfruro2)